<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.04/IR_04_extract_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pandas numpy torch transformers datasets scikit-learn tqdm

In [ ]:
# get the data from the github repository
aj_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/A_J_word.csv?raw=true"
bbc_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/BBC_word.csv?raw=true"
jp_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/J_P_word.csv?raw=true"
nyt_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/NYT_word.csv?raw=true"

In [ ]:
import re


def clean_text(text):
    # Normalize all types of single and double quotation marks to standard forms
    text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
    text = re.sub(r"[“”]", '"', text)  # Convert all double quote variations to "

    # remove any and all special characters since it will not be useful for our analysis
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    return text

In [ ]:
import pandas as pd

# load the data
aj_df = pd.read_csv(aj_url)
bbc_df = pd.read_csv(bbc_url)
jp_df = pd.read_csv(jp_url)
nyt_df = pd.read_csv(nyt_url)

In [ ]:
# clean the data
# aj_df['text'] = aj_df['text'].apply(clean_text)
# bbc_df['text'] = bbc_df['text'].apply(clean_text)
# jp_df['text'] = jp_df['text'].apply(clean_text)
# nyt_df['text'] = nyt_df['text'].apply(clean_text)

Here is the part you will need the model so I assume there is a variable called `model` which is the calssifier model you trained in the previous part.

In [ ]:
def extract_all_sentences(df):
    # this will return a dict with key the id of the article "aj_1" for example
    # and a list of all the sentences in the article

    all_sentences = {}

    for index, row in df.iterrows():
        text = row["document"]
        sentences = re.split(r"[.!?]", document)
        sentences = [sentence for sentence in sentences if sentence != ""]
        # clean the sentences
        sentences = [clean_text(sentence) for sentence in sentences]
        all_sentences[row['id']] = sentences

    return all_sentences

In [ ]:
# for each of the dataframes, extract all the sentences

aj_sentences = extract_all_sentences(aj_df)
bbc_sentences = extract_all_sentences(bbc_df)
jp_sentences = extract_all_sentences(jp_df)
nyt_sentences = extract_all_sentences(nyt_df)

# the sentences are now extracted and cleaned
# in the following format {id: [sentence1, sentence2, ...]}

In [ ]:
# now we wil use the model to classify each sentences for each article
# and we will classify the article based on the majority of the sentences

# TODO: load the model

# df is the new dataframe with id, class1, ..., calss5, majority_class
# the classi will contain the number of sentences in the article that belong to that class

concat_df = {**aj_sentences, **bbc_sentences, **jp_sentences, **nyt_sentences}

df = pd.DataFrame(columns=["id", "class1", "class2", "class3", "class4", "class5", "majority_class"])

for article_id, sentences in concat_df.items():
    # classify the sentences
    # and add 1 to the class of the sentence
    pass

# for each row in df calculate the majority class
# if there is a tie, choose the first
for index, row in df.iterrows():
    max_class = 0
    max_class_count = 0
    for i in range(1, 6):
        if row[f"class{i}"] > max_class_count:
            max_class = i
            max_class_count = row

In [ ]:
# save df into a csv file for analysis

df.to_csv("classified_articles.csv", index=False)